In [2]:
import numpy as np
import pandas as pd

In [3]:
# 25. divi
# 年频率。公司今年支付股息但前一年没有，则为1，否则为0。

# 26. divo
# 年频率。公司今年没有支付股息但在前一年支付了，则为1，否则为0。

df1 = pd.read_csv('../../data/csmar/日_分配文件.csv', usecols=['证券代码', '除权(息)日期', '分配类型'])
df1 = df1[df1['分配类型'] == 'CA']
df1['会计期间'] = df1['除权(息)日期'].apply(lambda x: x[:5] + '12-31')
df1 = df1.drop_duplicates(subset=['证券代码', '会计期间']).reset_index().drop(columns=['index'])
year_df = pd.DataFrame([f'{y}-12-31' for y in range(1990, 2023)], columns=['会计期间'])
stk_df = pd.read_csv('../../data/csmar/公司文件.csv')
stk_df['on'] = 0
year_df['on'] = 0
df2 = pd.merge(stk_df, year_df, on=['on'], how='left')[['证券代码', '会计期间']]
df3 = pd.merge(df2, df1, on=['证券代码', '会计期间'], how='left')
df3['divi'] = np.where((~df3['除权(息)日期'].isna()) & df3.groupby('证券代码').shift(1)['除权(息)日期'].isna(), 1, 0)
df3['divo'] = np.where(df3['除权(息)日期'].isna() & (~df3.groupby('证券代码').shift(1)['除权(息)日期'].isna()), 1, 0)
df4 = df3[['证券代码', '会计期间', 'divi', 'divo']]
df4.to_csv('../../output/csmar/年_25_26.csv', index=False, encoding='utf-8-sig')
df4

,证券代码,会计期间,divi,divo
0,1,1990-12-31,0,0
1,1,1991-12-31,1,0
2,1,1992-12-31,0,0
3,1,1993-12-31,0,0
4,1,1994-12-31,0,0
...,...,...,...,...
180835,900957,2018-12-31,0,0
180836,900957,2019-12-31,0,0
180837,900957,2020-12-31,0,0
180838,900957,2021-12-31,0,0
